# Kensho Kuremoto
## HW07-Part4 tnwb

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import json_normalize

url = "https://www.tnwb.uscourts.gov/Search/search.aspx?zoom_query=Car"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

results = soup.select("div.result_block, div.result_altblock")

In [3]:

data = []

for result in results:
    # name and URL
    title_tag = result.find("a")
    case_name = title_tag.get_text(strip=True) if title_tag else "N/A"
    case_url = title_tag["href"] if title_tag and title_tag.has_attr("href") else "N/A"

    # Category
    category_tag = result.find("span", class_="category")
    category = category_tag.get_text(strip=True).strip("[]") if category_tag else "N/A"

    # context
    context_tag = result.find("div", class_="context")
    context = context_tag.get_text(strip=True) if context_tag else "N/A"

    # additional details
    info_tag = result.find("div", class_="infoline")
    info = info_tag.get_text(strip=True) if info_tag else "N/A"

    data.append({
        "Case name": case_name,
        "URL": case_url,
        "Category": category,
        "Additional details": info
    })

df = pd.json_normalize(data)

In [4]:
df.head(10)

,Case name,URL,Category,Additional details
0,JDL: 04-24318 Jacquelline D. Black,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,Judges' Opinions,Terms matched: 1 - 102k - URL: https://ww...
1,WHB: 95-26401 Mary Lucy Cooper,https://www.tnwb.uscourts.gov/Opinions/whb/pdf...,Judges' Opinions,Terms matched: 1 - 27k - URL: https://www...
2,GHB: 97-12368 Billy G. Woffard,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,Terms matched: 1 - 71k - URL: https://www...
3,JDL: 97-30580 Mary Chrlis Hurst,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,Judges' Opinions,Terms matched: 1 - 32k - URL: https://www...
4,MRH: 20-20967 Jacob Braxton Herring 20-00094,https://www.tnwb.uscourts.gov/Opinions/mrh/pdf...,Judges' Opinions,Terms matched: 1 - 303k - URL: https://ww...
5,GHB: 95-11365 Melissa L. Bryan,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,Terms matched: 1 - 52k - URL: https://www...
6,JDL: 09-20339 Diane M. Miller,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,Judges' Opinions,Terms matched: 1 - 92k - URL: https://www...
7,GHB: 00-12340 Wanda K. Autry,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,Terms matched: 1 - 297k - URL: https://ww...
8,"GHB: 02-31651 Neil Bond Stewart, Jr. and Tina ...",https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,Terms matched: 1 - 291k - URL: https://ww...
9,GHB: 96-12039 Randy and Janice Willson,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,Terms matched: 1 - 201k - URL: https://ww...


In [5]:
df.to_csv("tnwb.uscourt.csv", index=False)
print("Saved successfully.")

Saved successfully.


# BONUS 1
## Split up the additional details into multiple columns

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

url = "https://www.tnwb.uscourts.gov/Search/search.aspx?zoom_query=Car"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

results = soup.select("div.result_block, div.result_altblock")

data = []

for result in results:
    title_tag = result.find("a")
    case_name = title_tag.get_text(strip=True) if title_tag else "N/A"
    case_url = title_tag["href"] if title_tag and title_tag.has_attr("href") else "N/A"

    category_tag = result.find("span", class_="category")
    category = category_tag.get_text(strip=True).strip("[]") if category_tag else "N/A"

    info_tag = result.find("div", class_="infoline")
    info_text = info_tag.get_text(strip=True) if info_tag else ""

    terms_matched = "N/A"
    size = "N/A"
    pdf_url = "N/A"

    match_tm = re.search(r"Terms matched:\s*(\d+)", info_text)
    if match_tm:
        terms_matched = match_tm.group(1)

    match_size = re.search(r"(\d+k)", info_text)
    if match_size:
        size = match_size.group(1)

    match_url = re.search(r"(https?://\S+\.pdf)", info_text)
    if match_url:
        pdf_url = match_url.group(1)

    data.append({
        "Case name": case_name,
        "URL": case_url,
        "Category": category,
        "terms matched": terms_matched,
        "size": size,
        "pdf URL": pdf_url
    })

newdf = pd.DataFrame(data)
newdf.head()

,Case name,URL,Category,terms matched,size,pdf URL
0,JDL: 04-24318 Jacquelline D. Black,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,Judges' Opinions,1,102k,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...
1,WHB: 95-26401 Mary Lucy Cooper,https://www.tnwb.uscourts.gov/Opinions/whb/pdf...,Judges' Opinions,1,27k,https://www.tnwb.uscourts.gov/Opinions/whb/pdf...
2,GHB: 97-12368 Billy G. Woffard,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,Judges' Opinions,1,71k,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...
3,JDL: 97-30580 Mary Chrlis Hurst,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,Judges' Opinions,1,32k,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...
4,MRH: 20-20967 Jacob Braxton Herring 20-00094,https://www.tnwb.uscourts.gov/Opinions/mrh/pdf...,Judges' Opinions,1,303k,https://www.tnwb.uscourts.gov/Opinions/mrh/pdf...


In [7]:
newdf.to_csv("tnwb.split.csv", index=False)
print("Saved successfully.")

Saved successfully.


# BONUS 2
## Download all of the PDFs of the cases

In [9]:
import os
import requests
import pandas as pd

pdf_dir = "pdfs"
os.makedirs(pdf_dir, exist_ok=True)

df = pd.read_csv("tnwb.split.csv")

for i, row in df.iterrows():
    pdf_url = row["pdf URL"]
    
    if pd.notna(pdf_url) and pdf_url.startswith("http"):
        filename = f"case_{i+1}.pdf"
        save_path = os.path.join(pdf_dir, filename)

        try:
            response = requests.get(pdf_url)
            response.raise_for_status()  
            with open(save_path, "wb") as f:
                f.write(response.content)
            print(f"Saved successfully : {save_path}")
        except Exception as e:
            print(f"error（{filename}）: {e}")


Saved successfully : pdfs/case_1.pdf
Saved successfully : pdfs/case_2.pdf
Saved successfully : pdfs/case_3.pdf
Saved successfully : pdfs/case_4.pdf
Saved successfully : pdfs/case_5.pdf
Saved successfully : pdfs/case_6.pdf
Saved successfully : pdfs/case_7.pdf
Saved successfully : pdfs/case_8.pdf
Saved successfully : pdfs/case_9.pdf
Saved successfully : pdfs/case_10.pdf
